In [1]:
from pyspark import SparkConf, SparkContext
import math, sys, glob, random
import sympy
import numpy as np
from nltk import ngrams, word_tokenize
import json
import matplotlib.pyplot as plt
import os
from itertools import *
# from tqdm.notebook import trange, tqdm
plt.style.use('ggplot')

conf = SparkConf().setMaster("local").setAppName("Final")
sc = SparkContext(conf=conf)

In [2]:
%%time
gossip = []
data_path = []
error = 0
success = 0
for root, dirs, files in os.walk("./data"):

    for file in files:
        with open(f'{root}/{file}') as f:
            try:
                d = json.load(f)
                d['author']
                d['messages'][0]
                if d['author'] == None:
                    error += 1
                    continue
#                 gossip.append(d)
#                 data_path.append(f'{root}/{file}')
                success += 1
            except:
                error += 1
#                 print(f'ERROR: {root}/{file}')



CPU times: user 4min 6s, sys: 1min 24s, total: 5min 31s
Wall time: 10min 12s


In [3]:
print(success)
print(error)

1731112
30918


In [4]:
# !cat ./data/all/politics/politics/20011215/M.1008356633.A.json

In [5]:
%%time
with open('HatePolitics_data.json', 'w', encoding='utf-8') as f:
    json.dump(gossip, f, ensure_ascii=False, indent=4)

CPU times: user 39.9 s, sys: 772 ms, total: 40.7 s
Wall time: 40.7 s


In [6]:
%%time
with open('HatePolitics_data_path.json', 'w', encoding='utf-8') as f:
    json.dump(data_path, f, ensure_ascii=False, indent=4)

CPU times: user 60 ms, sys: 20 ms, total: 80 ms
Wall time: 81.9 ms


In [ ]:
# %%time
# with open('./data.json') as f:
#     gossip = json.load(f)

In [ ]:
gossip[0].keys()

In [ ]:
gossip[0]['messages']

## Graph Construction

In [ ]:
all_user = {}
tag = {'推':1, '噓':-1, '→':0}

### comment : 1：贊, 0：沒情緒, -1：噓
# Mapper : (Author, [(push_user, comment), (push_user2, comment), ...])
def load_users(board):
    result = []
    for artical in board:
        if(artical['author'] not in all_user):
            all_user[artical['author']] = len(all_user)
        
        author = all_user[artical['author']]
        
        pusher_comment = []
        for push_artical in artical['messages']:
            if(push_artical['push_userid'] not in all_user):
                all_user[push_artical['push_userid']] = len(all_user)
            pusher_comment.append((all_user[push_artical['push_userid']], tag[push_artical['push_tag']]))
            
        result.append((author, pusher_comment))
    return result        

In [ ]:
%%time
gossip_graph = sc.parallelize(load_users(gossip))

In [ ]:
gossip_graph.collect()[0]

In [8]:
g = gossip_graph.flatMap(lambda x: [(x[0], (xx[0], xx[1])) for xx in x[1]])

In [9]:
g.collect()[22]

(8, (12, 1))

In [10]:
from operator import add
talk = g.map(lambda x: ((x[0], x[1][0]), x[1][1]))
talk2 = talk.reduceByKey(add)

In [11]:
len(talk.collect())

10731

In [12]:
len(talk2.collect())

6901

In [13]:
len(talk2.filter(lambda x: x[1] < 0).collect())

974